<a href="https://colab.research.google.com/github/Progressive-Programmer/image_to_ttf/blob/main/image_to_ttf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ----- Delete directory and files

In [2]:
import shutil
import os
import glob

files_and_dirs = glob.glob('*')
file_to_keep = 'input_image.jpeg'

for item in files_and_dirs:
    if item == file_to_keep:
        print(f"Skipping: {item} (kept as requested)")
        continue

    try:
        if os.path.isfile(item):
            os.remove(item)
            print(f"Deleted file: {item}")
        elif os.path.isdir(item):
            shutil.rmtree(item)
            print(f"Deleted directory: {item}")
        else:
            print(f"Skipping unknown item type: {item}")
    except Exception as e:
        print(f"Error deleting {item}: {e}")

Deleted directory: svg_family_cache
Deleted directory: sample_data


In [ ]:
# ----- 1. Start Here - Process the image file

In [4]:
import cv2
import numpy as np
import os

def process_font_grid_v2(image_path, rows=7, cols=6, target_size=512):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 21, 10)

    h_img, w_img = thresh.shape
    cell_h, cell_w = h_img // rows, w_img // cols

    output_folder = "svg_glyphs_input"
    os.makedirs(output_folder, exist_ok=True)

    margin_y = int(cell_h * 0.10)
    margin_x = int(cell_w * 0.10)

    char_idx = 0
    for r in range(rows):
        for c in range(cols):
            y1, y2 = r * cell_h + margin_y, (r + 1) * cell_h - margin_y
            x1, x2 = c * cell_w + margin_x, (c + 1) * cell_w - margin_x
            cell = thresh[y1:y2, x1:x2]

            cnts, _ = cv2.findContours(cell, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if cnts:
                x_c, y_c = [], []
                for cnt in cnts:
                    if cv2.contourArea(cnt) > 100:
                        bx, by, bw, bh = cv2.boundingRect(cnt)
                        x_c.extend([bx, bx + bw]); y_c.extend([by, by + bh])

                if x_c:
                    char_crop = cell[min(y_c):max(y_c), min(x_c):max(x_c)]

                    h_c, w_c = char_crop.shape
                    inner_size = int(target_size * 0.6)
                    scale = inner_size / max(h_c, w_c)
                    nw, nh = int(w_c * scale), int(h_c * scale)
                    resized = cv2.resize(char_crop, (nw, nh), interpolation=cv2.INTER_AREA)

                    canvas = np.zeros((target_size, target_size), dtype=np.uint8)
                    x_off, y_off = (target_size - nw) // 2, (target_size - nh) // 2
                    canvas[y_off:y_off+nh, x_off:x_off+nw] = resized

                    cv2.imwrite(f"{output_folder}/glyph_{char_idx:03d}.png", canvas)
                    char_idx += 1
    print(f"Cleaned {char_idx} glyphs with extra padding.")

process_font_grid_v2('input_image.jpeg')

Cleaned 42 glyphs with extra padding.


In [ ]:
# ----- 2. Second Step - Install libraries

In [6]:

!apt-get update
!apt-get install -y potrace fontforge

# Install python-level font libraries
!pip install fonttools[ufo,lxml] -q


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (4,160 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
potrace is already the newest version (1.16-2).
fo

In [ ]:
# ----- 3. Third Step - PIXEL and VECTOR Engine

In [7]:
import os
import subprocess
import cv2
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from fontTools.ttLib import TTCollection, TTFont
from google.colab import files

# --- CONFIG ---
input_png_folder = "svg_glyphs_input"
base_svg_dir = "svg_family_cache"
os.makedirs(base_svg_dir, exist_ok=True)

variants = [
    ("Thin",        "erode",  3, 0),
    ("Regular",     "none",   0, 0),
    ("Bold",        "dilate", 4, 0),
    ("Black",       "dilate", 9, 0),
    ("Italic",      "none",   0, 0.2), # Slant 0.2 for Italic
    ("BoldItalic",  "dilate", 4, 0.2)
]

# --- PIXEL & VECTOR ENGINE ---
def process_weight(variant):
    suffix, action, strength, slant = variant
    weight_svg_dir = os.path.join(base_svg_dir, suffix)
    os.makedirs(weight_svg_dir, exist_ok=True)

    png_files = sorted([f for f in os.listdir(input_png_folder) if f.endswith('.png')])

    for filename in png_files:
        img = cv2.imread(os.path.join(input_png_folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is None: continue

        _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

        if action != "none":
            kernel = np.ones((3,3), np.uint8)
            if action == "dilate":
                binary = cv2.dilate(binary, kernel, iterations=strength)
            else:
                binary = cv2.erode(binary, kernel, iterations=strength)

        padded = cv2.copyMakeBorder(binary, 60, 60, 60, 60, cv2.BORDER_CONSTANT, value=0)

        final_for_potrace = cv2.bitwise_not(padded)

        temp_bmp = f"temp_{suffix}_{filename}.bmp"
        cv2.imwrite(temp_bmp, final_for_potrace)

        name = os.path.splitext(filename)[0]
        svg_path = os.path.join(weight_svg_dir, f"{name}.svg")
        subprocess.run(["potrace", "--svg", "--flat", "-t", "50", temp_bmp, "-o", svg_path])
        os.remove(temp_bmp)

    # --- FONT ASSEMBLY SCRIPT ---
    otf_path = f"LogoFont_{suffix}.ttf"
    ff_script = f"""
import fontforge
import os
font = fontforge.font()
font.familyname, font.fontname = "LogoFont", "LogoFont-{suffix}"
svg_dir = "{weight_svg_dir}"
svg_files = sorted([f for f in os.listdir(svg_dir) if f.endswith('.svg')])
chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789!@#$%^&*()-_=+"

for i, filename in enumerate(svg_files):
    if i >= len(chars): break
    glyph = font.createChar(ord(chars[i]))
    glyph.importOutlines(os.path.join(svg_dir, filename))

    if {slant} != 0:
        glyph.transform([1, 0, {slant}, 1, 0, 0])

    glyph.simplify()
    glyph.transform([0.75, 0, 0, 0.75, 0, 0])
    bbox = glyph.boundingBox()
    glyph.transform([1, 0, 0, 1, -bbox[0] + (1000-(bbox[2]-bbox[0]))/2, -bbox[1]]) # Center glyph
    glyph.width = 1000
font.generate("{otf_path}")
"""
    with open(f"build_{suffix}.py", "w") as f: f.write(ff_script)
    subprocess.run(["fontforge", "-script", f"build_{suffix}.py"], capture_output=True)
    return otf_path

# --- EXECUTION ---
print("🚀 Executing Parallel Build for 6 variants...")
with ProcessPoolExecutor() as executor:
    generated_files = list(executor.map(process_weight, variants))

# --- STITCH & DOWNLOAD ---
print("🧵 Stitching into Family Collection...")
collection = TTCollection()
for f in generated_files:
    if os.path.exists(f): collection.fonts.append(TTFont(f))
collection.save("LogoFont_Complete_Family.ttc")
files.download("LogoFont_Complete_Family.ttc")
print("✅ Done! Your full font family is ready.")

🚀 Executing Parallel Build for 6 variants...
🧵 Stitching into Family Collection...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Done! Your full font family is ready.


In [ ]:
# -----  End - If the code worked your ttf file is downloaded.